In [2]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import numpy as np

In [5]:
# step 1, load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)
print(mnist.train.labels.shape)
print(mnist.train.images.shape)
print(mnist.test.labels.shape)
print(mnist.test.images.shape)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 10)
(55000, 784)
(10000, 10)
(10000, 784)


In [ ]:
def weights_variable(shape):
    W = tf.get_variable(initial=tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    b = tf.get_variable(initial=tf.constant(0.1, shape=shape))
    
def conv2d(x, W):
    tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
x_input = [None, 784]
y_input = [None, 10]